In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from subprocess import check_output

import re
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Verificar si CUDA está disponible
if torch.cuda.is_available():
    print("GPU detectada:", torch.cuda.get_device_name(0))
else:
    print("CUDA no está disponible. PyTorch está utilizando la CPU.")

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [ ]:
!python train.py --dataroot "C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\datasets\\DS-manseg-4seq" --name dualseq-basic-v1 --model pix2pix --direction AtoB --display_id -1 --input_nc 3 --output_nc 3

In [ ]:
def load_average_losses_from_log(log_path):
    """Carga y calcula la media de las pérdidas por época desde un archivo de logs."""
    epoch_losses = {}  # Diccionario {epoch: [lista de pérdidas]}
    
    with open(log_path, "r") as file:
        for line in file:
            # Extraer el número de época
            epoch_match = re.search(r"epoch: (\d+)", line)
            if not epoch_match:
                continue  # Si no hay epoch en la línea, saltar
            
            epoch = int(epoch_match.group(1))
            # Si el epoch aún no tiene registros, inicializar
            if epoch not in epoch_losses:
                epoch_losses[epoch] = { 
                    "G_GAN": [], "G_L1": [], "D_real": [], "D_fake": [], "D_t1_real": [], "D_t1_fake": [],
                    "G2_GAN": [], "G2_L1": [], "D2_real": [], "D2_fake": [], "D2_t1_real": [], "D2_t1_fake": []
                }

            # Extraer valores de pérdida (solo si epoch fue encontrado)
            for key in epoch_losses[epoch].keys():
                match = re.search(rf"{key}: ([\d\.]+)", line)
                if match:
                    epoch_losses[epoch][key].append(float(match.group(1)))

    # Calcular medias por época
    epochs = list(epoch_losses.keys())
    avg_losses = {key: [np.mean(epoch_losses[epoch][key]) for epoch in epochs] for key in epoch_losses[epochs[0]].keys()}

    return epochs, avg_losses

log_path = "C:\\Users\\Jesus\Documents\\TFM\\PROYECTO4\\checkpoints\\dualseq-basic-v1\\loss_log.txt"
epochs, losses = load_average_losses_from_log(log_path)
fig, axes = plt.subplots(2, 1, figsize=(10, 10))

# Generadores
axes[0].plot(epochs, losses["G_GAN"], label="G_GAN")
axes[0].plot(epochs, losses["G_L1"], label="G_L1")
axes[0].plot(epochs, losses["G2_GAN"], label="G2_GAN")
axes[0].plot(epochs, losses["G2_L1"], label="G2_L1")
axes[0].set_xlabel("Epochs", fontsize=16)
axes[0].set_ylabel("Losses", fontsize=16)
axes[0].set_title("Generators Losses", fontsize=18)
axes[0].legend(fontsize=14)
axes[0].tick_params(axis='both', labelsize=15)
axes[0].spines[['top', 'right']].set_visible(False)

# Discriminadores
axes[1].plot(epochs, losses["D_real"], label="D_real")
axes[1].plot(epochs, losses["D_fake"], label="D_fake")
axes[1].plot(epochs, losses["D2_real"], label="D2_real")
axes[1].plot(epochs, losses["D2_fake"], label="D2_fake")

axes[1].plot(epochs, losses["D_t1_real"], label="D_t1_real")
axes[1].plot(epochs, losses["D_t1_fake"], label="D_t1_fake")
axes[1].plot(epochs, losses["D2_t1_real"], label="D2_t1_real")
axes[1].plot(epochs, losses["D2_t1_fake"], label="D2_t1_fake")

axes[1].set_xlabel("Epochs", fontsize=16)
axes[1].set_ylabel("Losses", fontsize=16)
axes[1].set_title("Discriminators Losses", fontsize=18)
axes[1].legend(fontsize=14)
axes[1].tick_params(axis='both', labelsize=15)
axes[1].spines[['top', 'right']].set_visible(False)
    
plt.tight_layout()
plt.show()

### Plot loss functions

In [ ]:
with open(log_path, "r") as f:
    lines = f.readlines()

# Detectar la última época
epoch_numbers = [int(re.search(r'\(epoch: (\d+)', line).group(1)) for line in lines if "(epoch:" in line]
last_epoch = max(epoch_numbers)

# Filtrar líneas de la última época
last_epoch_lines = [line for line in lines if f"(epoch: {last_epoch}" in line]

# Métricas a extraer
metric_names = ["G_GAN", "G_L1", "D_real", "D_fake", "D2_real", "D2_fake"]
metric_values = {name: [] for name in metric_names}

# Extraer valores
for line in last_epoch_lines:
    for name in metric_names:
        match = re.search(fr'{name}: ([\d\.]+)', line)
        if match:
            metric_values[name].append(float(match.group(1)))

# Calcular promedios
mean_metrics = {name: sum(vals)/len(vals) if vals else 0 for name, vals in metric_values.items()}

plt.figure(figsize=(10, 6))
plt.bar(mean_metrics.keys(), mean_metrics.values())
plt.title(f"Epoch {last_epoch} - Metrics Mean")
plt.ylabel("Mean value")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
!ls checkpoints/

In [ ]:
!python test.py --dataroot "C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO2\\datasets\\top_left" --direction AtoB --model pix2pix --name dualseq-bt_rg-v1 --results_dir ./results/dualseq-CSIC-top_left

### Sequence generator

In [ ]:
!python test_recur.py --dataroot "C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\datasets\\tl_330" --direction AtoB --model pix2pix --name dualseq-bt_rg-v1 --results_dir ./results/dualseq-CSIC430 --num_recursive_steps 430